# Initialization

In [ ]:
$TenantId = $global:TenantId
# connect once for all necessary scopes for this notebook
# Disconnect-Graph
$null = Connect-MgGraph -Scopes "Directory.AccessAsUser.All", "Policy.Read.All", "RoleManagement.ReadWrite.Directory" -TenantId $TenantId -ErrorAction Stop

# load functions
. "..\src\functions.ps1"

# Privileged administration

## Limit the number of Global Administrators to less than 5

*Severity*: High

*Guid*: 9e6efe9d-f28f-463b-9bff-b5080173e9fe

In [ ]:

$DirectoryRoleId = Get-MgDirectoryRole -Filter "DisplayName eq 'Global Administrator'" | Select-Object -ExpandProperty Id

$GAAssigned = Get-MgDirectoryRoleMember -DirectoryRoleId  $DirectoryRoleId  

# get users that are eligble for GA
$GARoleDefinitionId = '62e90394-69f5-4237-9190-012177145e10'
[array]$EligebleGAPrincipals = Get-MgBetaRoleManagementDirectoryRoleEligibilityScheduleInstance -Filter "roleDefinitionId eq '$GARoleDefinitionId'" | Select-Object -ExpandProperty PrincipalId

$GAEligeble = Get-GroupMembers -DirectoryObjectByIds $EligebleGAPrincipals | Select-Object -Property Id, @{ Name = 'displayName'; Expression = { $_.AdditionalProperties.displayName } }, @{ Name = 'userPrincipalName'; Expression = { $_.AdditionalProperties.userPrincipalName } }

$Setting = ($GAAssigned + $GAEligeble) | Sort-Object -Unique -Property Id
$Compliant = $Setting.Count -lt 5

if($Compliant)
{
    Write-Host "Compliant to control; there are $($Setting.Count) Global Administrators (Assigned and Eligeble)" -ForegroundColor Green
}
else {
    Write-Host "Not compliant to control; there are $($Setting.Count) Global Administrators (Assigned and Eligeble)" -ForegroundColor Red
}


# External Identities

## Guest invite settings

*Severity*: High

*Guid*: be64dd7d-f2e8-4bbb-a468-155abc9164e9

External Collaboration Settings: Guest invite settings set to `'Only users assigned to specific admin roles can invite guest users'` or `'No one in the organization can invite guest users including admins (most restrictive)'`

In [ ]:
$AuthorizationPolicy = Get-MgPolicyAuthorizationPolicy

$Setting = $AuthorizationPolicy.AllowInvitesFrom
$Compliant = $Setting -in 'adminsAndGuestInviters', 'none'

if($Compliant)
{
    Write-Host "Compliant to control, setting is $($Setting)" -ForegroundColor Green
}
else {
    Write-Host "Not compliant to control, setting is $($Setting)" -ForegroundColor Red
}

## Guest user access restrictions

*Severity*: High

*Guid*: 459c373e-7ed7-4162-9b37-5a917ecbe48f

External Collaboration Settings: Guest user access set to `'Guest user access is restricted to properties and memberships of their own directory objects (most restrictive)'`

In [ ]:
# TODO: does not say anything about guest user access....

$ExternalIdentityPolicy = Get-MgBetaPolicyExternalIdentityPolicy #-ExpandProperty "AdditionalProperties"

# $ExternalIdentityPolicy | fl *
# $ExternalIdentityPolicy.AdditionalProperties | fl *



# User Setting

## User role permissions (Application registration)

*Severity*: High

*Guid*: a2cf2149-d013-4a92-9ce5-74dccbd8ac2a

Users can register applications should be set to `No`.

Users should not be allowed to register applications. Use specific roles such as `Application Developer`.

In [ ]:
$AuthorizationPolicy = Get-MgPolicyAuthorizationPolicy -Property "DefaultUserRolePermissions"

$Setting = $AuthorizationPolicy.DefaultUserRolePermissions.AllowedToCreateApps
$Compliant = $Setting -eq $false

if($Compliant)
{
    Write-Host "Compliant to control; users are not allowed to create applications" -ForegroundColor Green
}
else {
    Write-Host "Not compliant to control; users are allowed to create applications" -ForegroundColor Red
}

# Custom Domains

## Verified Domains

*Severity*: High

*Guid*: bade4aad-1e8c-439e-a946-667313c00567

Only validated customer domains are registered

In [ ]:
$Domains = Get-MgBetaDomain

$UnverifiedDomains = $Domains | Where-Object {-not $_.IsVerified}

$Setting = $UnverifiedDomains
$Compliant = $Setting.Count -eq 0

if($Compliant)
{
    Write-Host "Compliant to control; All domains are verified" -ForegroundColor Green
}
else {
    Write-Host "Not compliant to control; There are unverified domains registered: $($Setting | Select-Object -ExpandProperty Id)" -ForegroundColor Red
}